# Save and restore models
Source: https://www.tensorflow.org/tutorials/keras/save_and_restore_models
For Tensorflow2.0 : https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/keras/save_and_restore_models.ipynb#scrollTo=Pc9e6G6w1AWG

In [1]:
!pip install -q h5py pyyaml 

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
print(tf.__version__)

2.0.0-alpha0


In [6]:
(train_images, train_labels), (test_images, test_labels)= tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

test_images = test_images[:1000].reshape(-1, 28*28)/255.0
train_images = train_images[:1000].reshape(-1, 28*28)/255.0

## Define a Model

In [7]:
#Returns a short Sequential Model
def create_model():
    model = tf.keras.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu,input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    
    return model


###  Create Model Instance

In [8]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Checkpoint Callback usage

In [11]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                               save_weights_only=True,
                                               verbose = 1)

In [12]:
model = create_model()
model.fit(train_images, train_labels,
          epochs=10,
          validation_data = (test_images, test_labels),
         callbacks = [cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 960/1000 [===========================>..] - ETA: 0s - loss: 1.1814 - accuracy: 0.6521
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 3s 3ms/sample - loss: 1.1537 - accuracy: 0.6610 - val_loss: 0.7298 - val_accuracy: 0.7700
Epoch 2/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.4271 - accuracy: 0.8664
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.4245 - accuracy: 0.8690 - val_loss: 0.5443 - val_accuracy: 0.8290
Epoch 3/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.2787 - accuracy: 0.9234
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.2782 - accuracy: 0.9240 - val_loss: 0.4862 - val_accuracy: 0.8530
Epoch 4/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.1939 - accuracy: 0.9537
Epoch 00004

In [18]:
#this Creates a single collection of tensorflow checkpoints files that are updated at the end od every epoch.
!dir {checkpoint_dir}

 Volume in drive C is Win 10 Pro
 Volume Serial Number is FCFC-5707

 Directory of C:\Users\HP\Desktop\Code\TENSORFLOW\2 Tensorflow-Tutorials\1 Keras\5 Save and Restore Model\training_1

14-03-2019  14:37    <DIR>          .
14-03-2019  14:37    <DIR>          ..
14-03-2019  14:37                71 checkpoint
14-03-2019  14:37         4,889,870 cp.ckpt.data-00000-of-00001
14-03-2019  14:37             1,579 cp.ckpt.index
               3 File(s)      4,891,520 bytes
               2 Dir(s)  366,335,111,168 bytes free


#### Create a New Empty Model

In [19]:
# Now create a new empty model and see the difference

model = create_model()
loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model,accuracy: {:5.2f}% ".format(100*acc))

1000/1000 [==============================] - 1s 540us/sample - loss: 2.3857 - accuracy: 0.0840
Untrained model,accuracy:  8.40% 


#### Load Weights in New Model

In [21]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print("RESTORE and TRAINED model, accuracy: {:5.2f}% ".format(100*acc))

1000/1000 [==============================] - 0s 270us/sample - loss: 0.4194 - accuracy: 0.8700
RESTORE and TRAINED model, accuracy: 87.00% 


### Checkpoint callback options (save weights every 5-epochs)

In [22]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                verbose = 1,
                                                save_weights_only=True,
                                                #save weights every 5-epochs
                                                period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
         epochs = 50,
         callbacks=[cp_callback],
         validation_data=(test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [24]:
#now look at the resulting checkpoints
!dir {checkpoint_dir}

 Volume in drive C is Win 10 Pro
 Volume Serial Number is FCFC-5707

 Directory of C:\Users\HP\Desktop\Code\TENSORFLOW\2 Tensorflow-Tutorials\1 Keras\5 Save and Restore Model\training_2

14-03-2019  15:20    <DIR>          .
14-03-2019  15:20    <DIR>          ..
14-03-2019  15:20                81 checkpoint
14-03-2019  15:19         1,631,611 cp-0000.ckpt.data-00000-of-00001
14-03-2019  15:19               706 cp-0000.ckpt.index
14-03-2019  15:19         4,889,898 cp-0005.ckpt.data-00000-of-00001
14-03-2019  15:19             1,579 cp-0005.ckpt.index
14-03-2019  15:19         4,889,898 cp-0010.ckpt.data-00000-of-00001
14-03-2019  15:19             1,579 cp-0010.ckpt.index
14-03-2019  15:19         4,889,898 cp-0015.ckpt.data-00000-of-00001
14-03-2019  15:19             1,579 cp-0015.ckpt.index
14-03-2019  15:19         4,889,898 cp-0020.ckpt.data-00000-of-00001
14-03-2019  15:19             1,579 cp-0020.ckpt.index
14-03-2019  15:19         4,889,898 cp-0025.ckpt.data-00000-of-00001


In [26]:
# Check the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

### Test 

In [27]:
model = create_model()
model.load_weights(latest)
loss,acc = model.evaluate(test_images, test_labels)

print("RESTORED MODEL, Accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 1s 541us/sample - loss: 0.5040 - accuracy: 0.8710
RESTORED MODEL, Accuracy: 87.10%


## Save the Entire Model

### As an HDF5 file

In [33]:
model = create_model()
model.fit(train_images, train_labels, epochs=5,verbose=1)

#Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1301 - accuracy: 0.6830
Epoch 2/5
1000/1000 [==============================] - 1s 704us/sample - loss: 0.4120 - accuracy: 0.8900
Epoch 3/5
1000/1000 [==============================] - 1s 722us/sample - loss: 0.2830 - accuracy: 0.9230
Epoch 4/5
1000/1000 [==============================] - 1s 776us/sample - loss: 0.2058 - accuracy: 0.9530
Epoch 5/5
1000/1000 [==============================] - 1s 736us/sample - loss: 0.1544 - accuracy: 0.9660


### Recreate Same model from that file

In [35]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Check its Accuracy

In [37]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 308us/sample - loss: 0.4198 - accuracy: 0.8650
Restored model, accuracy: 86.50%


This technique saves everything:
- The weight values
- The model's configuration(architecture)
- The optimizer configuration

##  As a Saved Model

In [39]:
# Build a fresh model
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1695 - accuracy: 0.6600
Epoch 2/5
1000/1000 [==============================] - 1s 780us/sample - loss: 0.4309 - accuracy: 0.8670
Epoch 3/5
1000/1000 [==============================] - 1s 759us/sample - loss: 0.2821 - accuracy: 0.9300
Epoch 4/5
1000/1000 [==============================] - 1s 743us/sample - loss: 0.2086 - accuracy: 0.9480
Epoch 5/5
1000/1000 [==============================] - 1s 736us/sample - loss: 0.1537 - accuracy: 0.9650 - loss: 0.1572 - ac


# From here see: https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/keras/save_and_restore_models.ipynb#scrollTo=sq8fPglI1RWA

### Create a saved model

In [54]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

W0314 16:46:18.518616  2884 tf_logging.py:161] Export includes no default signature!
W0314 16:46:20.662527  2884 tf_logging.py:161] Export includes no default signature!


'./saved_models/1552562174'

## Saved models are placed in a time-stamped directory

In [51]:
!dir saved_models

 Volume in drive C is Win 10 Pro
 Volume Serial Number is FCFC-5707

 Directory of C:\Users\HP\Desktop\Code\TENSORFLOW\2 Tensorflow-Tutorials\1 Keras\5 Save and Restore Model\saved_models

14-03-2019  16:26    <DIR>          .
14-03-2019  16:26    <DIR>          ..
14-03-2019  16:26    <DIR>          assets
14-03-2019  16:26           264,712 saved_model.pb
14-03-2019  16:26    <DIR>          variables
               1 File(s)        264,712 bytes
               4 Dir(s)  366,262,358,016 bytes free


## Reload a fresh keras model from the saved model.

In [55]:
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.predict(test_images).shape

(1000, 10)

In [57]:
# The model has to be compiled before evaluating.
# This step is not required if the saved model is only being deployed.

new_model.compile(optimizer=model.optimizer,  # keep the optimizer that was loaded
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Evaluate the restored model.
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 1s 596us/sample - loss: 0.4173 - accuracy: 0.8640
Restored model, accuracy: 86.40%
